# Water Pump Classification

## Introduction

### Problem Statement

Using data from Taarifa and the Tanzanian Ministry of Water, we have been tasked with predicting which water pumps are functional, which need some repairs and which don't work at all.

This project is based on a competition hosted by Driven Data.

### Business Value

This project will help the Tanzanian Ministry of Water with their ongoing mission of providing access to safe drinking water and sanitation services to all.

Understanding which waterpoints are likely to fail and which features are the best predictors will help the Tanzanian government improve maintenance operations. 

### Methodology

We will adopt the OSEMN data science workflow, which involves:
1. Obtain (import the data)
2. Scrub (clean the data, deal with missing values and data types)
3. Explore (answer descriptives questions using EDA)
4. Model (build our predictive model)
5. iNterpret (comment on our model and findings)


## Obtain Data

### Import Libraries

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.preprocessing import LabelEncoder, StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB

from sklearn.metrics import roc_curve, auc, confusion_matrix, accuracy_score, f1_score, \
precision_score, recall_score, roc_auc_score

from imblearn.pipeline import make_pipeline, Pipeline
from imblearn.over_sampling import SMOTE

# do not show any warnings
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None) # prevents abbreviation (with '...') of columns

/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


### Load Data

The data has been provided and can be found here: https://www.drivendata.org/competitions/7/pump-it-up-data-mining-the-water-table/data/

In [2]:
# Load features DataFrame and preview
features_df = pd.read_csv('../Data/water-pump/training_set_values.csv', index_col = 'id')
features_df.head()

,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [3]:
# Get number of rows and columns
features_df.shape

(59400, 39)

In [4]:
# Load labels DataFrame and preview
labels_df = pd.read_csv('../Data/water-pump/training_set_labels.csv', index_col = 'id')
labels_df.head()

,status_group
id,
69572,functional
8776,functional
34310,functional
67743,non functional
19728,functional


In [5]:
# Get number of rows and columns
labels_df.shape

(59400, 1)

Let us double-check that the rows between the features and the labels match. 

In [6]:
# Use NP's assert_array_equal to check row indices match
np.testing.assert_array_equal(features_df.index.values, labels_df.index.values)

We have also been provided with a test set for competition purposes, which we will save for our final submission.

In [7]:
# Load test DataFrame
test_df = pd.read_csv('../Data/water-pump/test_set_values.csv')

### Data Dictionary

We have been provided with the following information regarding the columns of the `features` DataFrame.

* `amount_tsh` - Total static head (amount water available to waterpoint)
* `date_recorded` - The date the row was entered
* `funder` - Who funded the well 
* `gps_height` - Altitude of the well
* `installer` - Organization that installed the well
* `longitude` - GPS coordinate
* `latitude` - GPS coordinate
* `wpt_name` - Name of the waterpoint if there is one
* `num_private` -
* `basin` - Geographic water basin
* `subvillage` - Geographic location
* `region` - Geographic location
* `region_code` - Geographic location (coded)
* `district_code` - Geographic location (coded)
* `lga` - Geographic location
* `ward` - Geographic location
* `population` - Population around the well
* `public_meeting` - True/False
* `recorded_by` - Group entering this row of data
* `scheme_management` - Who operates the waterpoint
* `scheme_name` - Who operates the waterpoint
* `permit` - If the waterpoint is permitted
* `construction_year` - Year the waterpoint was constructed
* `extraction_type` - The kind of extraction the waterpoint uses
* `extraction_type_group` - The kind of extraction the waterpoint uses
* `extraction_type_class` - The kind of extraction the waterpoint uses
* `management` - How the waterpoint is managed
* `management_group` - How the waterpoint is managed
* `payment` - What the water costs
* `payment_type` - What the water costs
* `water_quality` - The quality of the water
* `quality_group` - The quality of the water
* `quantity` - The quantity of water
* `quantity_group` - The quantity of water
* `source` - The source of the water
* `source_type` - The source of the water
* `source_class` - The source of the water
* `waterpoint_type` - The kind of waterpoint
* `waterpoint_type_group` - The kind of waterpoint

We have been provided with the following information regarding the columns of the `labels` DataFrame:

* `functional` - the waterpoint is operational and there are no repairs needed
* `functional needs repair` - the waterpoint is operational, but needs repairs
* `non functional` - the waterpoint is not operational

## Scrub Data

In this section we will clean the data. This will involve dealing with missing values and ensuring the data types are correct. We may also look to create new features by combining existing features or extracting details from existing features.

### Investigate Data Types

In [8]:
# Get datatypes using info
features_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59400 entries, 69572 to 26348
Data columns (total 39 columns):
amount_tsh               59400 non-null float64
date_recorded            59400 non-null object
funder                   55765 non-null object
gps_height               59400 non-null int64
installer                55745 non-null object
longitude                59400 non-null float64
latitude                 59400 non-null float64
wpt_name                 59400 non-null object
num_private              59400 non-null int64
basin                    59400 non-null object
subvillage               59029 non-null object
region                   59400 non-null object
region_code              59400 non-null int64
district_code            59400 non-null int64
lga                      59400 non-null object
ward                     59400 non-null object
population               59400 non-null int64
public_meeting           56066 non-null object
recorded_by              59400 non-null obj

Overall, the data types appear correct.

We will transform `date_recorded` into a datetime object and extract year and month.

Where we have binary attributes such as `permit` and `public_meeting` we will transform these into 0/1 values instead of strings.

In [9]:
# Look at values for permit
features_df['permit'].value_counts(dropna = False)

True     38852
False    17492
NaN       3056
Name: permit, dtype: int64

In [10]:
# Use map to change True/False to 1/0. Note that this will also change NaN values to 0.
features_df['permit'] = features_df['permit'].map(lambda x: 1 if x == True else 0)

In [11]:
# Look at values for public_meeting
features_df['public_meeting'].value_counts(dropna = False)

True     51011
False     5055
NaN       3334
Name: public_meeting, dtype: int64

In [12]:
# Use map to change True/False to 1/0. Note that this will also change NaN values to 0.
features_df['public_meeting'] = features_df['public_meeting'].map(lambda x: 1 if x == True else 0)

### Fill Missing Values

In [13]:
# Get number of missing values
features_df.isna().sum().sort_values(ascending = False)[:10]

scheme_name          28166
scheme_management     3877
installer             3655
funder                3635
subvillage             371
population               0
ward                     0
lga                      0
district_code            0
region_code              0
dtype: int64

We will look at the 5 features with missing values and find the optimal method for imputation.

**scheme_name**

In [14]:
# Get number of unique values
features_df['scheme_name'].nunique()

2696

Due to the large number of missing entries and the variety of unique values this feature can take, there does not appear to be a reasonable value for imputation. Let us remove this feature.

In [15]:
# Drop scheme_name feature
features_df = features_df.drop('scheme_name', axis = 1)

**scheme_management**

In [16]:
# Get value counts
features_df['scheme_management'].value_counts(dropna = False)

VWC                 36793
WUG                  5206
NaN                  3877
Water authority      3153
WUA                  2883
Water Board          2748
Parastatal           1680
Private operator     1063
Company              1061
Other                 766
SWC                    97
Trust                  72
None                    1
Name: scheme_management, dtype: int64

We will keep the top 5 values and group all other values (including NaN) into an "other" category.

In [17]:
# Define function for scheme management
def scheme_management(value):
    ''' Keep top 5 values and assign rest to other, convert to lower case'''
    top_five = ['VWC', 'WUG', 'Water authority', 'WUA', 'Water Board']
    if value not in top_five:
        value = 'other' 
    value = value.lower()
    return value

# Apply function to df
features_df['scheme_management'] = features_df['scheme_management'].map(lambda x: scheme_management(x) )

**installer**

In [18]:
# Get value counts
features_df['installer'].value_counts(dropna = False)

DWE               17402
NaN                3655
Government         1825
RWE                1206
Commu              1060
                  ...  
M                     1
GRUMENTI              1
TMN                   1
Nu                    1
Jackson Makore        1
Name: installer, Length: 2146, dtype: int64

Similar to the previous feature, we will keep the top 5 values and group all other values (including NaN) into an "other" category.

In [19]:
# Get top 5 values excluding NaN
features_df['installer'].value_counts()[:5]

DWE           17402
Government     1825
RWE            1206
Commu          1060
DANIDA         1050
Name: installer, dtype: int64

In [20]:
features_df['installer'].value_counts()[:5]

DWE           17402
Government     1825
RWE            1206
Commu          1060
DANIDA         1050
Name: installer, dtype: int64

In [21]:
# Define function for installer
def installer(value):
    ''' Keep top 5 values and assign rest to other, convert to lower case'''
    top_five = ['DWE', 'Government', 'RWE', 'Commu', 'DANIDA']
    if value not in top_five:
        value = 'other'
    value = value.lower()
    return value

# Apply function to df
features_df['installer'] = features_df['installer'].map(lambda x: installer(x))

**funder**

In [22]:
# Get value counts
features_df['funder'].value_counts(dropna = False)

Government Of Tanzania    9084
NaN                       3635
Danida                    3114
Hesawa                    2202
Rwssp                     1374
                          ... 
Care/dwe                     1
Misheni                      1
Mwanaisha Mwidadi            1
Tanzaling                    1
Mp Mloka                     1
Name: funder, Length: 1898, dtype: int64

Similar to the previous feature, we will keep the top 5 values and group all other values (including NaN) into an "other" category.

In [23]:
# Get top 5 values excluding NaN
features_df['funder'].value_counts()[:5]

Government Of Tanzania    9084
Danida                    3114
Hesawa                    2202
Rwssp                     1374
World Bank                1349
Name: funder, dtype: int64

In [24]:
# Define function for funder
def funder(value):
    ''' Keep top 5 values and assign rest to other, convert to lower case'''
    top_five = ['Government Of Tanzania', 'Danida', 'Hesawa', 'Rwssp', 'World Bank']
    if value not in top_five:
        value = 'other'
    value = value.lower()
    return value

# Apply function to df
features_df['funder'] = features_df['funder'].map(lambda x: funder(x))

**subvillage**

In [29]:
# Get value counts
features_df['subvillage'].value_counts(dropna = False)

Madukani     508
Shuleni      506
Majengo      502
Kati         373
NaN          371
            ... 
Shimamo        1
Kanyange       1
Manyiriri      1
Kwemikwe       1
Kitae          1
Name: subvillage, Length: 19288, dtype: int64

With 19,288 unique values and no clear dominating values, it is difficult to determine the best course of action. We could fill the missing values with 'unknown' but is likely that there are just too many values for this categorical feature. We will drop this feature for now.

In [30]:
# Drop subvillage feature
features_df = features_df.drop('subvillage', axis = 1)

Let us check that all missing values (NaN) have been addressed.

In [32]:
# Check for NaN
features_df.isna().any().any()

False

### Deal with Placeholder Values

Whilst the previous section addressed excplicit missing values identified as such by the `isna()` method, let us investigate the data further and check for placeholders.

In [35]:
# Check for placeholder characters
features_df.isin(['?', '#', 'NaN', 'null', 'nan', 'N/A', '-']).sum().any()

False

Let us check that `0` is not being used as a placeholder.

In [41]:
# Check for 0
features_df.isin([0]).sum().sort_values(ascending = False)[:10]

num_private          58643
amount_tsh           41639
population           21381
construction_year    20709
permit               20548
gps_height           20438
public_meeting        8389
longitude             1812
district_code           23
region_code              0
dtype: int64

We will adddress these columns individually.

**num_private**

With almost all entries being `0` and no description of this column, there is not much we can do. Let us drop this feature.

In [42]:
# drop num_private
features_df = features_df.drop('num_private', axis = 1)

**amount_tsh**

## Explore Data